In [2]:
%matplotlib nbagg

import load
import pickle
import util
import matplotlib.pyplot as plt
import numpy as np
import collections as col
import random
from sklearn.cross_validation import train_test_split


In [3]:
#review = load.shuffle_data()
#pickle.dump(review,open("processed_text.p","wb"))
#pickle.dump(labels,open("label_data.p","wb"))
X = pickle.load(open("processed_text.p", "rb" ))
#y = pickle.load(open("label_data.p","rb"))

In [4]:
train,test = train_test_split(X,test_size=0.25, random_state=42)


### Sparse Representation of Data

#### Construct Complete Dictionary excluding stop words (words appearing less than 3 times across the corpus)

In [5]:
full_dict = col.Counter()
for k in train:
    for elem in k: 
        full_dict[elem] = full_dict[elem] + 1


In [6]:
def bow_of_example(example):
    c = col.Counter()
    for elem in range(len(example) -1):
        c[example[elem]] = c[example[elem]] + 1

    return c


In [7]:
i = 0
k = []
for example in train:
    k.append(bow_of_example(example))
    i = i + 1


In [8]:
i = 0
y = []
for example in train:
    y.append(example[len(example)-1])
    i = i + 1

print len(y)

1500


### Pegasos Algorithm

In [9]:
def pegasos(lamda, w, X_shuf,y_shuf,max_epochs):
    epoch = 0
    t = 0
    w_old = col.Counter()
    while epoch <= max_epochs:
        #data = list(zip(k,y))
        #random.shuffle(data)
        #X_shuf,y_shuf = zip(*data) 
        for j in range(len(X_shuf)):
            t = t+1
            eta_t = 1/(lamda*t)
            if (y_shuf[j]*util.dotProduct(w,X_shuf[j]) < 1):
                util.increment(w,-(eta_t*lamda), w)
                util.increment(w,eta_t*y_shuf[j],X_shuf[j])
            else:
                util.increment(w,-(eta_t*lamda), w)
        epoch = epoch + 1
    return w


In [10]:
def pegasos_1(lamda, w, X_shuf,y_shuf,max_epochs):
    epoch = 0
    t = 0
    w_old = col.Counter()
    while epoch <= max_epochs:
        #data = list(zip(k,y))
        #random.shuffle(data)
        #X_shuf,y_shuf = zip(*data) 
        j = 0
        for x in X_shuf:
            t = t+1
            eta_t = 1/(lamda*t)
            if (y_shuf[j]*util.dotProduct(w,x) < 1):
                util.increment(w,-(eta_t*lamda), w)
                util.increment(w,eta_t*y_shuf[j],x)
            else:
                util.increment(w,-(eta_t*lamda), w)
            j = j + 1
        epoch = epoch + 1
    return w

In [11]:
#print k[0]
import time
w = col.Counter()
#w = pegasos(0.01,w,k,y,5)
%load_ext line_profiler
%lprun -f pegasos_1 pegasos_1(0.01, w, k, y, 1)

print time.clock()
#print len(l)
pickle.dump(w,open("weight.p","wb"))

411.25281


In [19]:
def pegasos_fast(lamda, w, X_shuf,y_shuf,max_epochs):
    epoch = 0
    t = 1
    s = 1
    W_rep = col.Counter()
    w = col.Counter()
    util.increment(w,s,W_rep)
    #w = s * W_rep.values()
    while epoch <= max_epochs:
        for j in range(len(X_shuf)):
            t = t+1
            eta_t = 1/(lamda*t)
            if (y_shuf[j]*util.dotProduct(w,X_shuf[j]) < 1):
                s = (1 - (eta_t*lamda))*s
                if (s == 0):
                    s = 1
                    W_rep = col.Counter()
                    w.clear()
                    continue
                util.increment(W_rep,(1/s)*(eta_t*y_shuf[j]),X_shuf[j])
                w.clear()
                util.increment(w,s,W_rep)
        epoch = epoch + 1
        
    return w

In [20]:
#print k[0]
import time
w = col.Counter()
%lprun -f pegasos_fast pegasos_fast(0.01, w, k, y, 5)
#w = pegasos_fast(0.01,w,k,y,5)
print time.time()
#print w
pickle.dump(w,open("weight.p","wb"))

1456437153.03
